In [1]:
import pandas as pd
import datetime
import os

In [2]:
start_datetime = '2019-06-01'

In [3]:
# datetime_range = ['20190601']
datetime_range = [str(datetime.date()).replace('-', '')
                  for datetime in pd.date_range(start_datetime, periods=30, freq='D')]

In [4]:
path_prefix = 'data/tb_bus_user_usage/'
path_postfix = '_tb_bus_user_usage.csv'

result_postfix = '_정류장별시간대별노선별유형별승차인원분석.csv'

In [5]:
for path_infix in datetime_range:
    df = pd.read_csv(path_prefix + path_infix + path_postfix,
                     usecols=['route_id', 'route_no', 'geton_datetime', 
                              'geton_station_id', 'geton_stataion_name', 'user_type', 'user_count'],
#                               'geton_station_id', 'user_type', 'user_count'],
                     encoding='utf-8')
    
    df.loc[:, 'geton_datetime'] = df['geton_datetime'].astype(str)
    
    df.loc[:, 'geton_datetime'] = df['geton_datetime'].apply(lambda datetime: pd.to_datetime(datetime))
    
    df.loc[:, 'geton_hour'] = [date.hour for date in df['geton_datetime']]
    
    df.loc[:, 'geton_minute'] = [int(date.minute/30)/2 for date in df['geton_datetime']]
    # (예) 35분 -> 0.5, 22분 -> 0으로 환산.
    
    df.loc[:, 'geton_hour'] = df.loc[:, 'geton_hour'] + df.loc[:, 'geton_minute']
    
    del df['geton_datetime']
    del df['geton_minute']
    
    grouped = df.groupby(by=['geton_station_id', 'geton_stataion_name', 'route_id', 'route_no', 'user_type', 'geton_hour'], 
                         as_index=False)
    grouped[['user_count']].sum().to_csv(path_infix + result_postfix, encoding='ms949')
    
print('batch done...')